In [3]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
from torch.utils import data

In [2]:
# transform = transforms.Compose(
#         [transforms.Resize((224,224)),transforms.ToTensor(),
#          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


# all_data=torchvision.datasets.ImageFolder(root=r"C:\Users\jimmy\fullData",transform=transform)

# batch_size = 100

# train_split = 0.7 # 70% training data

# indices = list(range(len(all_data)))
# split = int(np.floor(train_split * len(all_data)))

# # split out the training data
# train_indices, val_n_test_indices = indices[split:], indices[:split]

# # split out the validation and testing data
# val_split = 0.5 # 15% validation data 15% testing data
# split2 = int(np.floor(val_split * len(all_data)-split))
# val_indices, test_indices = indices[split2:], indices[:split2]

# data_train = data.Subset(all_data, train_indices)
# data_validation = data.Subset(all_data, val_indices)
# data_test = data.Subset(all_data, test_indices)


In [3]:

# import cv2

# img_raw=cv2.imread(r"C:\Users\jimmy\Labelled_Data\anger\S010_004_00000007.png")
# tensor= torch.from_numpy(img_raw/255)

# print(tensor.size())


In [4]:
import torchvision.models
alexnet = torchvision.models.alexnet(pretrained=True)

In [3]:
data_dir = 'C:/Users/jimmy/Desktop/Labelled_Data/'
train_dir = os.path.join(data_dir, 'Training/')
val_dir = os.path.join(data_dir, 'Validation/')

data_train = datasets.ImageFolder(train_dir, transform=transform_train)
data_validation = datasets.ImageFolder(val_dir, transform=transform_train)

# print out some data stats
print('Num training images: ', len(data_train))
print('Num validation images: ', len(data_validation))

NameError: name 'os' is not defined

In [5]:
# def img_to_feature(data,batch_size):
#     features = []
#     label = []
#     loader = torch.utils.data.DataLoader(data, batch_size=batch_size,shuffle=True)
#     for imgs, labels in iter(loader):
#         features.append(alexnet.features(imgs))
#         label.append(labels)
#     return features,label

In [5]:
# features_val ,label_val = img_to_feature(data_validation,1000)
# features_train,label_train = img_to_feature(data_train,1000)
#features_test,label_test = img_to_feature(test_data,50)


# saved_val_features = []
# saved_val_labels = []

# saved_test_features = []
# saved_test_labels = []

# saved_train_features = []
# saved_train_labels = []

# for i in range(0,len(features_val)):
#     saved_val_features.append(features_val[i])
#     saved_val_labels.append(label_val[i])

# for i in range(0,len(features_train)):
#     saved_train_features.append(features_train[i])
#     saved_train_labels.append(label_train[i])

# torch.save(saved_val_features,'features_val_bs50.pt')
# torch.save(saved_val_labels,'label_val_bs50.pt')

# torch.save(saved_train_features,'features_train_bs50.pt')
# torch.save(saved_train_labels,'label_train_bs50.pt')


In [5]:
class CNN_transfer(nn.Module):
    def __init__(self):
        super(CNN_transfer, self).__init__()
        self.name = "net_transfer"
        self.conv1 = nn.Conv2d(256, 256, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(256*3*3, 20)
        self.fc2 = nn.Linear(20, 7)
        
        
    def forward(self, x):
        #based on the layers defined above (they are in order)
        #between each layer, it needs an activation function in between to 
        #generate output from upper layer, this output will be the input to the
        #lower layer

        x = self.pool(F.relu(self.conv1(x)))
        #print(x.shape)
        x = x.view(-1, 256*3*3)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = x.squeeze(1) # Flatten to [batch_size]
        return x

In [21]:
def get_model_name(name, batch_size, learning_rate, epoch,n):
    path = "model_{0}_bs{1}_lr{2}_epoch{3}_n={4}".format(name,batch_size,learning_rate,epoch,n)
    return path

def get_accuracy_transfer(model, train=False,batch_size=1000):
    if train:
        data = torch.load('C:/Users/jimmy/features_train_bs128.pt')
        label = torch.load('C:/Users/jimmy/label_train_bs128.pt')
        
    else:
        data = torch.load('features_val_bs200.pt')
        label = torch.load('label_val_bs200.pt')
        

    correct = 0
    total = 0
    for i in range(0,len(data)):
        output = model(data[i]) # We don't need to run F.softmax
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(label[i].view_as(pred)).sum().item()
        total += data[i].shape[0]
    return correct / total

def train_transfer(model, batch_size=1000, num_epochs=20, learning_rate = 0.0006,weight_decay=0.0002):
    start_time = time.time()
    torch.manual_seed(1000)
    #train_loader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size=batch_size)
    train_loader = torch.load('features_train_bs128.pt')
    label_loader = torch.load('label_train_bs128.pt')
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay = weight_decay)
    

    training_curve, iters, losses, train_acc, val_acc = [], [], [], [], []

    # training
    n = 0 # the number of iterations
    x_value = 0
    for epoch in range(num_epochs):
        for i in range(0,len(train_loader)):
        #for imgs, labels in iter(train_loader):
            out = model(train_loader[i])             # forward pass
            loss = criterion(out, label_loader[i]) # compute the total loss
            loss.backward()               # backward pass (compute parameter updates)
            optimizer.step()              # make the updates for each parameter
            optimizer.zero_grad()         # a clean up step for PyTorch

            # save the current training information
            iters.append(n)
            losses.append(float(loss)/batch_size)             # compute *average* loss
            n += 1
        train_acc.append(get_accuracy_transfer(model, train=True,batch_size=batch_size)) # compute training accuracy 
        val_acc.append(get_accuracy_transfer(model, train=False,batch_size=batch_size))  # compute validation accuracy
            
        training_curve.append(x_value)
        x_value += 1
        #print(("Epoch {}: Train acc: {} |").format(epoch + 1,train_acc[-1]))

        print(("Epoch {}: Train acc: {} |"+"Validation acc: {}").format(epoch + 1,train_acc[-1],val_acc[-1]))
        model_path = get_model_name(model.name, batch_size, learning_rate, epoch,n)
        torch.save(model.state_dict(), model_path)

    # plotting
    plt.title("Training Curve")
    plt.plot(iters, losses, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(training_curve, train_acc, label="Train")
    plt.plot(training_curve, val_acc, label="Validation")
    plt.xlabel("Epochs")
    plt.ylabel("Training Accuracy")
    plt.legend(loc='best')
    plt.show()
    
    end_time = time.time()
    duration = end_time - start_time

    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))
    print ("Trained Duration: {} seconds".format(duration))

In [6]:
best_model = CNN_transfer()
best_model.load_state_dict(torch.load('model_net_transfer_bs128_lr2e-05_epoch48_n=882'), 
                           strict=False)

<All keys matched successfully>

In [7]:
def get_label(img):
    dataimg = alexnet.features(img)
    output = best_model(dataimg)
    pred = output.max(1, keepdim=True)[1]
    if pred[0][0] == 0:
        emote = 'anger'
    elif pred[0][0] == 1:
        emote = 'contempt'
    elif pred[0][0] == 2:
        emote = 'disgust'
    elif pred[0][0] == 3:
        emote = 'fear'
    elif pred[0][0] == 4:
        emote = 'happy'
    elif pred[0][0] == 5:
        emote = 'sadness'
    elif pred[0][0] == 6:
        emote = 'surprise'
    else:
        raise ValueError
    print(emote)
    return emote

In [8]:
import cv2
import PIL.Image as Image
from torchvision.transforms import ToTensor, ToPILImage
import torch.utils
import signal

In [33]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
import time
facesPath = 'C:\opencv\sources\data\haarcascades\haarcascade_frontalface_default.xml'

faceCascade = cv2.CascadeClassifier(facesPath)




def post_process2(img_raw):
    # post processing into cropped images of just the face
    gray = cv2.cvtColor(img_raw, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor= 1.1,
        minNeighbors=8,
        minSize=(55, 55),
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    for (x, y, w, h) in faces:
        cv2.rectangle(img_raw, (x, y), (x+w, y+h), (0, 0, 255), 2)
        img_crop = img_raw[y:y+h, x:x+w]
        img_crop = Image.fromarray(img_crop.astype('uint8'), 'RGB')
        transform = transforms.Compose(
                    [transforms.Resize((224,224)),transforms.ToTensor(),
                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        img_tens = transform(img_crop)
        img_tens = img_tens.unsqueeze(0)
        #cv2.imshow(img_raw)
        #img_small = img_raw
        emote = get_label(img_tens)
        if emote == 'anger':
            img_small = cv2.imread(r"C:\Users\jimmy\anger.png")
        elif emote == 'contempt':
            img_small = cv2.imread(r"C:\Users\jimmy\contempt.png")
        elif emote == 'disgust':
            img_small = cv2.imread(r"C:\Users\jimmy\disgust.png")
        elif emote == 'fear':
            img_small = cv2.imread(r"C:\Users\jimmy\fear.png")
        elif emote == 'happy':
            img_small = cv2.imread(r"C:\Users\jimmy\happy.png")
        elif emote == 'sadness':
            img_small = cv2.imread(r"C:\Users\jimmy\sadness.png")
        elif emote == 'surprise':
            img_small = cv2.imread(r"C:\Users\jimmy\surprise.png")
        else:
            img_small = cv2.imread(r"C:\Users\jimmy\neutral.png")
            
        img_small = cv2.cvtColor(img_small, cv2.COLOR_RGB2RGBA).copy()
        
        
        y1, y2 = y, y + img_small.shape[1]
        x1, x2 = x, x + img_small.shape[1]

        alpha_s = img_small[:, :, 3] / 255.0
        alpha_l = 1.0 - alpha_s
        print((alpha_s).shape)
        print((alpha_l).shape)
        print(img_small.shape)
        alpha_s*img_small[:,:,0] + alpha_l*img_raw[y1:y2, x1:x2, 0]
        

        for c in range(0, 3):
            img_raw[y1:y2, x1:x2, c] = (alpha_s * img_small[:, :, c] + alpha_l * img_raw[y1:y2, x1:x2, c])
            #img_raw[y1:y2, x1:x2, c] = alpha_l * img_raw[y1:y2, x1:x2, c]
        #img_raw = cv2.cvtColor(img_raw, cv2.COLOR_RGBA2RGB).copy()
            

output_notebook()

cap = cv2.VideoCapture(0)
ret, frame = cap.read()
frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA) # because Bokeh expects a RGBA image
frame=cv2.flip(frame, -1) # because Bokeh flips vertically
width=frame.shape[1]
height=frame.shape[0]
p = figure(x_range=(0,width), y_range=(0,height), output_backend="webgl", width=width, height=height)
myImage = p.image_rgba(image=[frame], x=0, y=0, dw=width, dh=height)
show(p, notebook_handle=True)
while True:
    ret, frame = cap.read()
    post_process2(frame)
    frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
    frame=cv2.flip(frame, -1)
    
    myImage.data_source.data['image']=[frame]
    push_notebook()
    time.sleep(0.3)

Loading BokehJS ...

anger
(240, 240)
(240, 240)
(240, 240, 4)


ValueError: operands could not be broadcast together with shapes (240,240) (231,240) 